In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from gatubelxs.spark_utils import SparkUtils

In [14]:
import os 
base_path = "/home/jovyan/notebooks/data"

for topic in ["page_views", "click_events", "user_interactions"]:
    os.makedirs(f"{base_path}/{topic}/checkpoint", exist_ok=True)
    os.makedirs(f"{base_path}/{topic}", exist_ok=True)  # For the data files too

In [15]:
spark = SparkSession.builder \
    .appName("FinalProjectGatubelxs") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [16]:
TOPICS = ["page_views", "click_events", "user_interactions"]
KAFKA_BOOTSTRAP_SERVER = "kafka:9093"

In [17]:
page_views_schema = SparkUtils.generate_schema(
    [

        ("user_id", "string"),
        ("session_id", "string"),
        ("page_url", "string"),
        ("referrer_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string")
    ]
)

In [18]:
click_events_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("element_id", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string"),
        ("x_coord", "float"),
        ("y_coord", "float"),
    ]
)

In [19]:
user_interaction_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("interaction_type", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("details", "string"),
        ("timestamp", "string")
    ]
)

In [20]:
def create_stream_query(topic, schema, output_dir):
    kafka__stream = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
        .option("subscribe", topic) \
        .load()
    
    parsed_data = kafka__stream.selectExpr("CAST(value as STRING) as json") \
        .withColumn("data", from_json(col("json"), schema)) \
        .select("data.*")

    return parsed_data.writeStream \
        .format("parquet") \
        .outputMode("append") \
        .option("path", f"{base_path}/{topic}") \
        .option("checkpointLocation", f"{base_path}/{topic}/checkpoint") \
        .trigger(processingTime="5 seconds") \
        .start()


In [21]:
queries = [
    create_stream_query("page_views", page_views_schema, "./datalake/page_views"),
    create_stream_query("click_events", click_events_schema, "./datalake/click_events"),
    create_stream_query("user_interactions", user_interaction_schema, "./datalake/user_interactions"),
]

for q in queries:
    q.awaitTermination()

25/05/08 23:54:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 23:54:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 23:54:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/08 23:54:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/08 23:54:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/08 23:54:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied

KeyboardInterrupt: 

In [11]:
sc.stop()

25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/05/08 21:48:21 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 0, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/05/08 21:48:21 ERRO